In [33]:
import pandas as pd
import utils
import plotly.graph_objects as go 

In [34]:
pair = "EUR_USD"
granularity = "H1"

In [35]:
df = pd.read_pickle(utils.get_his_data_filename(pair,granularity)) 


In [81]:
non_cols = ['time','volume']
mod_cols = [ x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [83]:
df_ma = df[['time','mid_o','mid_h','mid_l','mid_c']].copy()

In [96]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8
7,2021-09-17T13:00:00.000000000Z,1.17736,1.17750,1.17506,1.17538,1.177546
8,2021-09-17T14:00:00.000000000Z,1.17537,1.17572,1.17344,1.17379,1.177115
9,2021-09-17T15:00:00.000000000Z,1.17378,1.17406,1.17288,1.17336,1.176621
10,2021-09-17T16:00:00.000000000Z,1.17338,1.17368,1.17247,1.17256,1.175976
11,2021-09-17T17:00:00.000000000Z,1.17258,1.17322,1.17246,1.17270,1.175286


In [89]:
df_ma['MA_8'] = df_ma.mid_c.rolling(window=8).mean()

In [95]:
df_ma.dropna(inplace=True)

In [92]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c,MA_8
0,2021-09-17T06:00:00.000000000Z,1.17704,1.17750,1.17688,1.17724,NaN
1,2021-09-17T07:00:00.000000000Z,1.17722,1.17850,1.17716,1.17731,NaN
2,2021-09-17T08:00:00.000000000Z,1.17730,1.17832,1.17684,1.17772,NaN
3,2021-09-17T09:00:00.000000000Z,1.17774,1.17878,1.17740,1.17822,NaN
4,2021-09-17T10:00:00.000000000Z,1.17820,1.17860,1.17787,1.17828,NaN
5,2021-09-17T11:00:00.000000000Z,1.17827,1.17886,1.17758,1.17884,NaN
6,2021-09-17T12:00:00.000000000Z,1.17886,1.17890,1.17727,1.17738,NaN
7,2021-09-17T13:00:00.000000000Z,1.17736,1.17750,1.17506,1.17538,1.177546
8,2021-09-17T14:00:00.000000000Z,1.17537,1.17572,1.17344,1.17379,1.177115
9,2021-09-17T15:00:00.000000000Z,1.17378,1.17406,1.17288,1.17336,1.176621


In [97]:
df_plot = df_ma.iloc[-100:].copy()

In [80]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1),opacity=1,
    increasing_fillcolor="#24A06B",
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color="#2EC886",
    decreasing_line_color="#FF3A4C",
))
fig.add_trace(go.Scatter(x=df_plot.time, y=df_plot.MA_8,line=dict(color="#027FC3",width=2),line_shape='spline',name='MA_8'))
fig.update_layout(
    width=1200,
    height=300,
    margin=dict(t=10,r=10,b=10,l=10),
    font=dict(size=10, color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,
    fixedrange=True,
    rangeslider=dict(visible=False) 
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True,
    fixedrange=True,
)
fig.show()